In [9]:
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import ipynb.fs.full.Myfunction as M
torch.set_grad_enabled(True)

In [10]:
img_train = M.Mydataset(file_path='/home/zqw/GalSim/a_mycode/data/train_data/',
                          label_path='/home/zqw/GalSim/a_mycode/data/train_data.txt')
train_loader = DataLoader(img_train, batch_size=100, shuffle=True, num_workers=0,
                          pin_memory=False)

## in cpu

In [3]:
def train(net, train_loader, loss_func, optimizer):
    total_loss = 0
    for i,batch in enumerate(train_loader): # Get Batch
        images,labels = M.batch_data(batch,train_loader.batch_size) 
        #images = images.to(device)
        #labels = labels.to(device)
        preds = net(images) # Pass Batch
        loss = loss_func(preds, labels)

        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step()
        if i % 10 == 0:
            print((i+1)*train_loader.batch_size,total_loss/(i+1))
        #print("[{0}]:{1}".format(i,loss))
    mean_loss = total_loss / 30
    return mean_loss

import matplotlib.pyplot as plt
def show_curve(losses, num_epochs):
    x = np.arange(num_epochs)
    y = np.array(losses)
    plt.plot(x, y, c='b')
    plt.axis()
    plt.xlabel('epoch')
    plt.ylabel('{}'.format(title))
    plt.show()
    
def fit(net, num_epochs,optimizer):
    losses = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch + 1, num_epochs))      
        loss = train(net, train_loader, loss_func, optimizer)
        print(loss)
        losses.append(loss)
    show_curve(losses, num_epochs)

In [4]:
#device = torch.device('cuda:0'if torch.cuda.is_available() else 'cpu')
import time
begin = time.time()
net = M.Net()
num_epochs = 10
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
fit(net,num_epochs,optimizer)
run_time = time.time() - begin
print(run_time)

Epoch 1/10:
100 4.726448059082031


KeyboardInterrupt: 

In [47]:
torch.save(net.state_dict(),'/home/zqw/pytorch/data/netparam_n.pkl')

In [8]:
np.arange(15).shape

(15,)

## run train loop

In [8]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import torchvision
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs
params = OrderedDict(lr = [.01] ,batch_size = [60,120,200])
runs = RunBuilder.get_runs(params)

In [16]:
m = RunManager() 
#device = torch.device('cuda:0'if torch.cuda.is_available() else 'cpu')
for run in RunBuilder.get_runs(params):
    net = Net()
    #device = torch.device(run.device)
    train_loader = DataLoader(img_train, batch_size=run.batch_size, shuffle=True, num_workers=0,
                             pin_memory=False)
    optimizer = torch.optim.Adam(net.parameters(), lr=run.lr)
    m.begin_run(run, net, train_loader)
    for epoch in range(6):
        m.begin_epoch()
        for batch in loader:
            images,labels = data(batch) 
            #images = images.to(device)
            #labels = labels.to(device)
            
            preds = net(images) # Pass Batch
            loss = nn.MSELoss(preds, labels)
            optimizer.zero_grad()
            loss.backward() # Calculate Gradients
            optimizer.step()
            
            m.track_loss(loss, batch)
        m.end_epoch()
    m.end_run()
m.save('results')

expected scalar type Double but found Float
Error occurs, No graph saved


RuntimeError: expected scalar type Double but found Float

In [15]:
class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.tb = None
    
    def begin_run(self, run, network, loader):

        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = net
        self.loader = train_loader
        self.tb = SummaryWriter(comment=f'-{run}')

        images, labels = next(iter(self.loader))

        self.tb.add_image('images')
        self.tb.add_graph(self.network, images)
    
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0
        
    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        #self.epoch_num_correct = 0
        
    def end_epoch(self):

        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss
        #accuracy = self.epoch_num_correct / len(self.loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        #self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
        
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        #results["accuracy"] = accuracy
        results['epoch duration'] = epoch_duration
        results['run duration'] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')
        clear_output(wait=True)
        display(df)
    
    def track_loss(self, loss, batch):
        self.epoch_loss += loss.item() * self.loader.batch_size
    

In [7]:
def save(self, fileName):

    pd.DataFrame.from_dict(
        self.run_data, orient='columns'
    ).to_csv(f'{fileName}.csv')

## in gpu cuda:0

In [11]:
def train(net, train_loader, loss_func, optimizer):
    total_loss = 0
    time.sleep(0.003)
    for i,batch in enumerate(train_loader): # Get Batch
        images,labels = M.batch_data(batch,train_loader.batch_size)
        images = images.cuda()
        labels = labels.cuda()
        
        preds = net(images) # Pass Batch
        loss = loss_func(preds, labels)
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step()
        if i % 10 == 0:
            print((i+1)*train_loader.batch_size,total_loss/(i+1))
        #print("[{0}]:{1}".format(i,loss))
    mean_loss = total_loss / 30
    return mean_loss

def fit(net, num_epochs,optimizer):
    losses = []
    with torch.no_grad():
        net = net.cuda()
    loss_func = nn.MSELoss().cuda()
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch + 1, num_epochs))
        loss = train(net, train_loader, loss_func, optimizer) 
        print(loss)
        losses.append(loss)
    #show_curve(losses, num_epochs)
        
begin = time.time()
net = M.Net()
net.load_state_dict(torch.load('/home/zqw/pytorch/data/netparam_n.pkl')) # 读取保存的参数
num_epochs = 2
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
fit(net,num_epochs,optimizer)
run_time = time.time() - begin
print('run_time:',run_time)

RuntimeError: CUDA error: an illegal memory access was encountered

In [8]:
# 保存参数
torch.save(net.state_dict(),'/home/zqw/pytorch/data/netparam_n.pkl')